# There is no need to run this file. It simply shows the procedure used to create the train, test, and val sets for documentation purposes.

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split

In [ ]:
# Load video_info
video_info = pd.read_csv("/datasets/cgn/EGOCOM/video_info.csv")

In [ ]:
# Create train, val, and test sets.
video_info['gender'] = video_info['speaker_gender'] == 'male'
video_info['cid'] = video_info['conversation_id'].apply(lambda x: x.split("__part")[0])
video_info['day'] = video_info['conversation_id'].apply(lambda x: int(x[4:5]))
convs = video_info.groupby(['cid']).mean().drop(['video_id', 'video_speaker_id', 'speaker_is_host', 'word_count'], axis = 1)
convs['duration_seconds'] = convs['duration_seconds'] > 500
convs['native_speaker'] = convs['native_speaker'] < 1
convs['gender'] = convs['gender'] < 1
convs['day'] = convs['day'] > 2
convs.drop([ 'background_music', 'gender', 'background_fan', 'duration_seconds'], axis = 1, inplace=True)
strats = ['background_fan', 'background_music', 'native_speaker', 'speaker_gender', ]
train_idx, test_idx = train_test_split(range(len(convs)), test_size = 0.2, stratify = convs, random_state = 0)
val_idx = [test_idx[0], test_idx[-1]]
test_idx = test_idx[1:-1]

In [ ]:
# Add information to video_info
video_info = pd.read_csv("/datasets/cgn/EGOCOM/video_info.csv")
video_info['cid'] = video_info['conversation_id'].apply(lambda x: x.split("__part")[0])
train = pd.DataFrame([(z, i in train_idx) for i, z in enumerate(list(convs.index))], columns = ['cid', 'train'])
val = pd.DataFrame([(z, i in val_idx) for i, z in enumerate(list(convs.index))], columns = ['cid', 'val'])
test = pd.DataFrame([(z, i in test_idx) for i, z in enumerate(list(convs.index))], columns = ['cid', 'test'])
video_info = pd.merge(video_info, train, on='cid')
video_info = pd.merge(video_info, val, on='cid')
video_info = pd.merge(video_info, test, on='cid')

# Make sure all videos are accounted for
assert(all(video_info[['train', 'val', 'test']].sum(axis = 1) == 1))

In [ ]:
# Store video_info
video_info.to_csv("/datasets/cgn/EGOCOM/video_info.csv", index = False)